# 10 分钟快速上手 fastNLP torch

&emsp;&emsp;在这个例子中，我们将使用 BERT 来解决 `conll2003` 数据集中的命名实体识别任务。

In [1]:
# Linux/Mac 下载数据，并解压
import platform
if platform.system() != "Windows":
    !wget https://data.deepai.org/conll2003.zip --no-check-certificate -O conll2003.zip
    !unzip -o conll2003.zip -d conll2003
# Windows用户请通过复制该url到浏览器下载该数据并解压

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (zh_CN.UTF-8)
--2022-08-03 09:35:11--  https://data.deepai.org/conll2003.zip
Connecting to 10.176.52.116:3333... connected.
Proxy request sent, awaiting response... 200 OK
Length: 982975 (960K) [application/x-zip-compressed]
Saving to: 'conll2003.zip'

conll2003.zip       100%[===================>] 959.94K   586KB/s    in 1.6s    

2022-08-03 09:35:15 (586 KB/s) - 'conll2003.zip' saved [982975/982975]

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (zh_CN.UTF-8)
Archive:  conll2003.zip
  inflating: conll2003/metadata      
  inflating: conll2003/test.txt      
  inflating: conll2003/train.txt     
  inflating: conll2003/valid.txt     


### 1.  数据加载
&emsp;&emsp;目前在 ``conll2003`` 目录下有 ``train.txt``, ``test.txt``与``valid.txt`` 三个文件，文件的格式为 [conll格式](https://universaldependencies.org/format.html)，其编码格式为 [BIO](https://blog.csdn.net/HappyRocking/article/details/79716212) 类型。可以通过继承 `fastNLP.io.Loader` 来简化加载过程，继承了 `Loader` 函数后，只需要在实现读取单个文件 `_load()` 函数即可。

In [2]:
from fastNLP import DataSet, Instance
from fastNLP.io import Loader


# 继承Loader之后，我们只需要实现其中_load()方法，_load()方法传入一个文件路径，返回一个fastNLP DataSet对象，其目的是读取一个文件。
class ConllLoader(Loader):
    def _load(self, path):
        ds = DataSet()
        with open(path, 'r') as f:
            segments = []
            for line in f:
                line = line.strip()
                if line == '':  # 如果为空行，说明需要切换到下一句了。
                    if segments:
                        raw_words = [s[0] for s in segments]
                        raw_target = [s[1] for s in segments]
                        # 将一个 sample 插入到 DataSet中
                        ds.append(Instance(raw_words=raw_words, raw_target=raw_target))  
                    segments = []
                else:
                    parts = line.split()
                    assert len(parts)==4
                    segments.append([parts[0], parts[-1]])
        return ds
    

# 直接使用 load() 方法加载数据集, 返回的 data_bundle 是一个 fastNLP.io.DataBundle 对象，该对象相当于将多个 dataset 放置在一起，
#  可以方便之后的预处理，DataBundle 支持的接口可以在 ！！！ 查看。
data_bundle = ConllLoader().load({
    'train': 'conll2003/train.txt',
    'test': 'conll2003/test.txt',
    'dev': 'conll2003/valid.txt'
})
"""
也可以通过 ConllLoader().load('conll2003/') 来读取，其原理是load()函数将尝试从'conll2003/'文件夹下寻找文件名称中包含了
'train'、'test'和'dev'的文件，并分别读取将其命名为'train'、'test'和'dev'（如文件夹中同一个关键字出现在了多个文件名中将导致报错，
此时请通过dict的方式传入路径信息）。但在我们这里的数据里，没有文件包含dev，所以无法直接使用文件夹读取，转而通过dict的方式传入读取的路径，
该dict的key也将作为读取的数据集的名称，value即对应的文件路径。
"""

print(data_bundle)  # 打印 data_bundle 可以查看包含的 DataSet 
# data_bundle.get_dataset('train')  # 可以获取单个 dataset

In total 3 datasets:
	train has 14987 instances.
	test has 3684 instances.
	dev has 3466 instances.



#### 2.  数据预处理
&emsp;&emsp;接下来，我们将演示如何通过 fastNLP 提供的 `apply` 函数方便快捷地进行预处理。我们需要进行的预处理操作有：  

1. 使用 `BertTokenizer` 将文本转换为 index；同时记录每个 word 被 bpe 之后第一个 bpe 的 index，用于得到 word 的 `hidden state`；  
2. 使用 [Vocabulary](../../api/generated/fastNLP.core.Vocabulary.rst) 来将 `raw_target` 转换为序号。  

In [3]:
# fastNLP 中提供了BERT, RoBERTa, GPT, BART 模型，更多的预训练模型请直接使用transformers
from fastNLP.transformers.torch import BertTokenizer
from fastNLP import cache_results, Vocabulary

# 使用cache_results来装饰函数，会将函数的返回结果缓存到'caches/{param_hash_id}_cache.pkl'路径中（其中{param_hash_id}是根据
#   传递给 process_data 函数参数决定的，因此当函数的参数变化时，会再生成新的缓存文件。如果需要重新生成新的缓存，(a) 可以在调用process_data
#   函数时，额外传入一个_refresh=True的参数; 或者（b）删除相应的缓存文件。此外，保存结果时，cache_results默认还会
#   记录 process_data 函数源码的hash值，当其源码发生了变动，直接读取缓存会发出警告，以防止在修改预处理代码之后，忘记刷新缓存。）
@cache_results('caches/cache.pkl')
def process_data(data_bundle, model_name):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    def bpe(raw_words):
        bpes = [tokenizer.cls_token_id]
        first = [0]
        first_index = 1  # 记录第一个bpe的位置
        for word in raw_words:
            bpe = tokenizer.encode(word, add_special_tokens=False)
            bpes.extend(bpe)
            first.append(first_index)
            first_index += len(bpe)
        bpes.append(tokenizer.sep_token_id)
        first.append(first_index)
        return {'input_ids': bpes, 'input_len': len(bpes), 'first': first, 'first_len': len(raw_words)}
    # 对data_bundle中每个dataset的每一条数据中的raw_words使用bpe函数，并且将返回的结果加入到每条数据中。
    data_bundle.apply_field_more(bpe, field_name='raw_words', num_proc=4)
    # 对应我们还有 apply_field() 函数，该函数和 apply_field_more() 的区别在于传入到 apply_field() 中的函数应该返回一个 field 的
    #   内容（即不需要用dict包裹了）。此外，我们还提供了 data_bundle.apply() ，传入 apply() 的函数需要支持传入一个Instance对象，
    #   更多信息可以参考对应的文档。

    # tag的词表，由于这是词表，所以不需要有padding和unk
    tag_vocab = Vocabulary(padding=None, unknown=None)
    # 从 train 数据的 raw_target 中获取建立词表
    tag_vocab.from_dataset(data_bundle.get_dataset('train'), field_name='raw_target')
    # 使用词表将每个 dataset 中的raw_target转为数字，并且将写入到target这个field中
    tag_vocab.index_dataset(data_bundle.datasets.values(), field_name='raw_target', new_field_name='target')

    # 可以将 vocabulary 绑定到 data_bundle 上，方便之后使用。
    data_bundle.set_vocab(tag_vocab, field_name='target')

    return data_bundle, tokenizer

data_bundle, tokenizer = process_data(data_bundle, 'bert-base-cased', _refresh=True)  # 第一次调用耗时较长，第二次调用则会直接读取缓存的文件
# data_bundle = process_data(data_bundle, 'bert-base-uncased')  # 由于参数变化，fastNLP 会再次生成新的缓存文件。

Output()

Output()

Output()

[09:35:38] INFO     Save cache to /remote-home/shxing/fnlp-doc/fastNLP/d ]8;id=171711;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/utils/cache_results.py\cache_results.py]8;;\:]8;id=961592;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/utils/cache_results.py#343\343]8;;\
                    ocs/source/tutorials/torch/caches/c7f74559_cache.pkl                     
                    .                                                                        

### 3. DataLoader  
&emsp;&emsp;由于现在的深度学习算法大都基于 mini-batch 进行优化，因此需要将多个 sample 组合成一个 batch 再输入到模型之中。在自然语言处理中，不同的 sample 往往长度不一致，需要进行 padding 操作。在 fastNLP 中，我们使用 `fastNLP.TorchDataLoader` 帮助用户快速进行 padding ，我们使用了 `fastNLP.Collator` 对象来进行 pad ，`Collator` 会在迭代过程中根据第一个 batch 的数据自动判定每个 field 是否可以进行 pad ，可以通过 `Collator.set_pad()` 函数修改某个 field 的 pad 行为。

In [4]:
from fastNLP import prepare_dataloader

# 将 data_bundle 中每个 dataset 取出并构造出相应的 DataLoader 对象。返回的 dls 是一个 dict ，包含了 'train', 'test', 'dev' 三个
#   fastNLP.TorchDataLoader 对象。
dls = prepare_dataloader(data_bundle, batch_size=24) 


# fastNLP 将默认尝试对所有 field 都进行 pad ，如果当前 field 是不可 pad 的类型，则不进行pad；如果是可以 pad 的类型
#   默认使用 0 进行 pad 。
for dl in dls.values():
    # 可以通过 set_pad 修改 padding 的行为。
    dl.set_pad('input_ids', pad_val=tokenizer.pad_token_id)
    # 如果希望忽略某个 field ，可以通过 set_ignore 方法。
    dl.set_ignore('raw_target')
    dl.set_pad('target', pad_val=-100)
# 另一种设置的方法是，可以在 dls = prepare_dataloader(data_bundle, batch_size=32) 之前直接调用 
#  data_bundle.set_pad('input_ids', pad_val=tokenizer.pad_token_id); data_bundle.set_ignore('raw_target')来进行设置。
#  DataSet 也支持这两个方法。
# 若此时调用 batch = next(dls['train'])，则 batch 是一个 dict ，其中包含了
#  'input_ids': torch.LongTensor([batch_size, max_len])
#  'input_len': torch.LongTensor([batch_size])
#  'first': torch.LongTensor([batch_size, max_len'])
#  'first_len': torch.LongTensor([batch_size])
#  'target': torch.LongTensor([batch_size, max_len'-2])
#  'raw_words': List[List[str]]  # 因为无法判断，所以 Collator 不会做任何处理

### 4. 模型准备
&emsp;&emsp;传入给fastNLP的模型，需要有两个特殊的方法 ``train_step``、``evaluate_step``，前者默认在 `fastNLP.Trainer` 中进行调用，后者默认在 fastNLP.Evaluator 中调用。如果模型中没有 ``train_step`` 方法，则Trainer会直接使用模型的``forward`` 函数；如果模型没有 ``evaluate_step`` 方法，则Evaluator会直接使用模型的 ``forward`` 函数。``train_step`` 方法（或当其不存在时，``forward`` 方法）的返回值必须为 dict 类型，并且必须包含 ``loss`` 这个 key 。

&emsp;&emsp;此外fastNLP会使用形参名匹配的方式进行参数传递，例如以下模型
```python
class Model(nn.Module):
   def train_step(self, x, y):
        return {'loss': (x-y).abs().mean()}
```
fastNLP 将尝试从 `DataLoader` 返回的 batch(假设包含的 `key` 为 `input_ids`, `target`) 中寻找 'x' 和 'y' 这两个 key ，如果没有找到则会报错。有以下的方法可以解决报错
- 修改 `train_step` 的参数为 `(input_ids, target)`，以保证和 `DataLoader` 返回的 batch 中的 key 匹配
- 修改 `DataLoader` 中返回 batch 的 key 的名字为 `(x, y)`
- 在 `Trainer` 中传入参数 `train_input_mapping={'input_ids': 'x', 'target': 'y'}` 将输入进行映射，`train_input_mapping` 也可以是一个函数，更多 `train_input_mapping` 的介绍可以参考文档。

``evaluate_step`` 也是使用同样的匹配方式，前两条解决方法是一致的，第三种解决方案中，需要在 `Evaluator` 中传入 `evaluate_input_mapping={'input_ids': 'x', 'target': 'y'}`。

In [5]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from fastNLP.transformers.torch import BertModel
from fastNLP import seq_len_to_mask
import torch.nn.functional as F


class BertNER(nn.Module):
    def __init__(self, model_name, num_class, tag_vocab=None):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.mlp = nn.Sequential(nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size),
                                nn.Dropout(0.3),
                                nn.Linear(self.bert.config.hidden_size, num_class))
        self.tag_vocab = tag_vocab  # 这里传入 tag_vocab 的目的是为了演示 constrined_decode 
        if tag_vocab is not None:
            self._init_constrained_transition()
    
    def forward(self, input_ids, input_len, first):
        attention_mask = seq_len_to_mask(input_len)
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        first = first.unsqueeze(-1).repeat(1, 1, last_hidden_state.size(-1))
        first_bpe_state = last_hidden_state.gather(dim=1, index=first)
        first_bpe_state = first_bpe_state[:, 1:-1]  # 删除 cls 和 sep
        
        pred = self.mlp(first_bpe_state)
        return {'pred': pred}
    
    def train_step(self, input_ids, input_len, first, target):
        pred = self(input_ids, input_len, first)['pred']
        loss = F.cross_entropy(pred.transpose(1, 2), target)
        return {'loss': loss}
    
    def evaluate_step(self, input_ids, input_len, first):
        pred = self(input_ids, input_len, first)['pred'].argmax(dim=-1)
        return {'pred': pred}
    
    def constrained_decode(self, input_ids, input_len, first, first_len):
        # 这个函数在推理时，将保证解码出来的 tag 一定不与前一个 tag 矛盾【例如一定不会出现 B-person 后面接着 I-Location 的情况】
        # 本身这个需求可以在 Metric 中实现，这里在模型中实现的目的是为了方便演示：如何在fastNLP中使用不同的评测函数
        pred = self(input_ids, input_len, first)['pred']
        cons_pred = []
        for _pred, _len in zip(pred, first_len):
            _pred = _pred[:_len]
            tags = [_pred[0].argmax(dim=-1).item()]  # 这里就不考虑第一个位置非法的情况了
            for i in range(1, _len):
                tags.append((_pred[i] + self.transition[tags[-1]]).argmax().item())
            cons_pred.append(torch.LongTensor(tags))
        cons_pred = pad_sequence(cons_pred, batch_first=True)
        return {'pred': cons_pred}
    
    def _init_constrained_transition(self):
        from fastNLP.modules.torch import allowed_transitions
        allowed_trans = allowed_transitions(self.tag_vocab)
        transition = torch.ones((len(self.tag_vocab), len(self.tag_vocab)))*-100000.0
        for s, e in allowed_trans:
            transition[s, e] = 0
        self.register_buffer('transition', transition)

model = BertNER('bert-base-uncased', len(data_bundle.get_vocab('target')), data_bundle.get_vocab('target'))

[09:37:51] WARNING  Some weights of the model checkpoint at            ]8;id=426572;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=76042;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1490\1490]8;;\
                    bert-base-uncased were not used when initializing                        
                    BertModel: ['cls.predictions.decoder.weight',                            
                    'cls.predictions.bias',                                                  
                    'cls.predictions.transform.LayerNorm.weight',                            
                    'cls.predictions.transform.dense.weight',                                
                    'cls.predictions.transform.LayerNorm.bias',                              
                    'cls.seq_relationship.bias',                                             
                    'cls.predictions.transform.dense.bias',                                  
                    'cls.seq_relationship.weight']                                           
                    - This IS expected if you are initializing                               
                    BertModel from the checkpoint of a model trained                         
                    on another task or with another architecture (e.g.                       
                    initializing a BertForSequenceClassification model                       
                    from a BertForPreTraining model).                                        
                    - This IS NOT expected if you are initializing                           
                    BertModel from the checkpoint of a model that you                        
                    expect to be exactly identical (initializing a                           
                    BertForSequenceClassification model from a                               
                    BertForSequenceClassification model).                                    

           INFO     All the weights of BertModel were initialized from ]8;id=9817;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=809135;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1507\1507]8;;\
                    the model checkpoint at bert-base-uncased.                               
                    If your task is similar to the task the model of                         
                    the checkpoint was trained on, you can already use                       
                    BertModel for predictions without further                                
                    training.                                                                

### 5. Trainer 的使用
&emsp;&emsp;fastNLP 的 `Trainer` 是用于对模型进行训练的部件。

In [6]:
from torch import optim
from fastNLP import Trainer, LoadBestModelCallback, TorchWarmupCallback
from fastNLP import SpanFPreRecMetric

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
callbacks = [
    LoadBestModelCallback(),   # 用于在训练结束之后加载性能最好的model的权重
    TorchWarmupCallback()
]

trainer = Trainer(model=model, train_dataloader=dls['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  # 在评测时将 dataloader 中的 first_len 映射 seq_len, 因为 SpanFPreRecMetric.update 接口需要输入一个名为 seq_len 的参数
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=0, monitor='f#f', fp16=False)  # fp16 为 True 的话，将使用 float16 进行训练。
trainer.run()

[09:37:53] INFO     Running evaluator sanity check for 2 batches.              ]8;id=551808;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=583832;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

+++++++++++++++++++++++++++++ Eval. results on Epoch:1, Batch:0 +++++++++++++++++++++++++++++

{
  "f#f": 0.38953,
  "pre#f": 0.45941,
  "rec#f": 0.338102
}

[09:38:47] INFO     The best performance for monitor f#f:0.38953 was  ]8;id=832668;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=453248;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    achieved in Epoch:1, Global Batch:625. The                               
                    evaluation result:                                                       
                    {'f#f': 0.38953, 'pre#f': 0.45941, 'rec#f':                              
                    0.338102}                                                                

           INFO     Loading best model from buffer with f#f:  ]8;id=486989;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=208507;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#115\115]8;;\
                    0.38953...                                                               

### 6. Evaluator的使用
&emsp;&emsp;fastNLP 中用于评测数据的对象。

In [7]:
from fastNLP import Evaluator
from fastNLP import SpanFPreRecMetric

evaluator = Evaluator(model=model, dataloaders=dls['test'], 
                      metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                      evaluate_input_mapping={'first_len': 'seq_len'}, 
                      device=0)
evaluator.run()

Output()

{'f#f': 0.360507, 'pre#f': 0.405441, 'rec#f': 0.32454}

{'f#f': 0.360507, 'pre#f': 0.405441, 'rec#f': 0.32454}

In [8]:
# 如果想评测一下使用 constrained decoding的性能，则可以通过传入 evaluate_fn 指定使用的函数
def input_mapping(x):
    x['seq_len'] = x['first_len']
    return x
evaluator = Evaluator(model=model, dataloaders=dls['test'], device=0,
                      metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))},
                      evaluate_fn='constrained_decode',
                      # 如果将 first_len 重新命名为了 seq_len, 将导致 constrained_decode 的输入缺少 first_len 参数，因此
                      #   额外重复一下 'first_len': 'first_len'，使得这个参数不会消失。
                      evaluate_input_mapping=input_mapping)
evaluator.run()

Output()

{'f#f': 0.394526, 'pre#f': 0.471007, 'rec#f': 0.339412}

{'f#f': 0.394526, 'pre#f': 0.471007, 'rec#f': 0.339412}

### 7. 其它

#### 7.1 使用多卡进行训练、评测

&emsp;&emsp;fastNLP 同样也支持分布式训练，并且提供了多种使用方式，您可以根据您的使用习惯选择适合您的方法。

&emsp;&emsp;最简单的方法就是将 `Trainer` 和 `Evaluator` 的 `device` 参数设置成多卡，然后直接运行即可。如果想要使用设备0, 1进行分布式训练，将 `device` 设置为 `[0,1]` 即可。

```python
trainer = Trainer(model=model, train_dataloader=dls['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=[0,1], monitor='f#f', fp16=False)  # 改动的地方只有 device
trainer.run()

evaluator = Evaluator(model=model, dataloaders=dls['test'], device=[0,1], # 改动的地方只有 device
                      metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))},
                      evaluate_fn='constrained_decode',
                      evaluate_input_mapping=input_mapping)
evaluator.run()
```

&emsp;&emsp;第二种方法则不需要对代码做出任何改变，然后通过 `python -m torch.distributed.run` 启动分布式训练。注意，通过该方法启动时，关于gpu设备的分配将完全按照命令行的参数进行，代码中 `device` 的值会被 fastNLP 忽略。详情可以查阅 pytorch 关于分布式训练的说明。

&emsp;&emsp;第三种方法则更加接近 pytorch 分布式训练原本的语法。您可以在使用 Trainer 之前初始化分布式中的通信组，然后使用 `DistributedDataParallel` 包裹模型，最后通过指令 `python -m torch.distributed.run` 进行分布式训练。这种使用方式依据 pytorch 分布式训练的流程，更加适合此前经常使用分布式训练的用户。不过需要注意的是，您需要为 Trainer 和 Evaluator 指定 `data_device` 参数来告诉 fastNLP 应该将数据迁移到哪个设备上。

```python
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel

# 初始化通信组
dist.init_process_group()
# 分布式模型
model = DistributedDataParallel(model)
# 找到每个 rank 对应的设备
local_rank = int(os.environ['LOCAL_RANK'])
local_device = torch.device(f"cuda:{local_rank}")

trainer = Trainer(model=model, train_dataloader=dls['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=None, data_device=local_device, monitor='f#f', fp16=False)  # device 参数会被忽略，并且需要指定 data_device
trainer.run()
```

&emsp;&emsp;#### 7.2 使用ZeRO优化

在最新版本的 torch 1.12 中，pytorch 加入了 `fully sharded data parallel` 的并行策略，对标微软 `deepspeed` 提出的 ZeRO 优化，帮助我们节省训练中的内存。fastNLP也加入了该功能，只需要将 `driver` 参数指定为 `'torch_fsdp'` 即可，其它的使用方法则和上文提到的 **分布式训练** 相似。您可以查阅 [fastNLP 关于 fsdp 的说明](../../api/generated/fastNLP.core.TorchFSDPDriver.rst) 和 [pytorch 的官方教程](https://pytorch.org/tutorials/intermediate/FSDP_tutorial.html) 来进行更加深入的了解。

```python
trainer = Trainer(model=model, driver='torch_fsdp', train_dataloader=dls['train'], # 指定 driver
                  optimizers=optimizer, evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=[0,1], monitor='f#f', fp16=False)
trainer.run()
```

#### 7.3 通过overfit测试快速验证模型

&emsp;&emsp;在训练模型时我们往往难以验证模型和数据的正确性，以及各种训练参数的设置也不好判断。fastNLP 中 Trainer 提供的 `overfit_batches` 参数可以帮您简单地进行验证。指定该参数后，fastNLP 会将训练集中的 `overfit_batches` 个 batch （如果为-1则为全部）同时作为此次训练集和验证集开始训练，即训练集和验证集都是同一个数据集。如果一切设置正常，那么训练的结果应该在数次迭代之后趋于过拟合（如在分类任务中准确率会达到95%以上甚至100%）。如果结果并不理想，就需要考虑是否有数据中存在矛盾、学习率选择不当、模型结构不当等问题了。

In [9]:
# 重新初始化
model = BertNER('bert-base-uncased', len(data_bundle.get_vocab('target')), data_bundle.get_vocab('target'))
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
callbacks = [
    TorchWarmupCallback()
]

trainer = Trainer(model=model, train_dataloader=dls['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=30, evaluate_every=-10, # 每 10 个 epoch 查看一次
                  evaluate_input_mapping={'first_len': 'seq_len'},
                  device=0, monitor='f#f', fp16=False, overfit_batches=10)  # 在训练集的前 10 个 batch 上进行过拟合验证
trainer.run() # 最终结果会趋近于 1

[09:39:42] WARNING  Some weights of the model checkpoint at            ]8;id=735419;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=171018;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1490\1490]8;;\
                    bert-base-uncased were not used when initializing                        
                    BertModel: ['cls.predictions.decoder.weight',                            
                    'cls.predictions.bias',                                                  
                    'cls.predictions.transform.LayerNorm.weight',                            
                    'cls.predictions.transform.dense.weight',                                
                    'cls.predictions.transform.LayerNorm.bias',                              
                    'cls.seq_relationship.bias',                                             
                    'cls.predictions.transform.dense.bias',                                  
                    'cls.seq_relationship.weight']                                           
                    - This IS expected if you are initializing                               
                    BertModel from the checkpoint of a model trained                         
                    on another task or with another architecture (e.g.                       
                    initializing a BertForSequenceClassification model                       
                    from a BertForPreTraining model).                                        
                    - This IS NOT expected if you are initializing                           
                    BertModel from the checkpoint of a model that you                        
                    expect to be exactly identical (initializing a                           
                    BertForSequenceClassification model from a                               
                    BertForSequenceClassification model).                                    

           INFO     All the weights of BertModel were initialized from ]8;id=121799;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=89897;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1507\1507]8;;\
                    the model checkpoint at bert-base-uncased.                               
                    If your task is similar to the task the model of                         
                    the checkpoint was trained on, you can already use                       
                    BertModel for predictions without further                                
                    training.                                                                

[09:39:43] INFO     Running evaluator sanity check for 2 batches.              ]8;id=667061;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=768833;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

...

++++++++++++++++++++++++++++ Eval. results on Epoch:20, Batch:0 +++++++++++++++++++++++++++++

{
  "f#f": 0.695444,
  "pre#f": 0.657596,
  "rec#f": 0.737913
}

++++++++++++++++++++++++++++ Eval. results on Epoch:30, Batch:0 +++++++++++++++++++++++++++++

{
  "f#f": 0.988564,
  "pre#f": 0.98731,
  "rec#f": 0.989822
}

[09:40:06] INFO     The best performance for monitor f#f:0.988564 was ]8;id=605298;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=30799;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    achieved in Epoch:30, Global Batch:300. The                              
                    evaluation result:                                                       
                    {'f#f': 0.988564, 'pre#f': 0.98731, 'rec#f':                             
                    0.989822}                                                                

#### 7.4 复杂Monitor的使用

&emsp;&emsp;fastNLP 的 Trainer 含有一个参数 `monitor` ，该参数表示在训练中会检测该参数代表的值并按条件执行相应功能。需要注意的是，仅当用户传入的 `Callback` 需要该参数 `Callback` 的 `monitor` 为 `None` 时，Trainer 中的 `monitor` 才会起作用。具体而言，在上文训练 conll 数据集的过程中，我们传入了 `LoadBestModelCallback`，它的功能就是在 `monitor` 变得更好时保存模型，并在训练结束后将这个最好的模型加载回来。在训练过程中，fastNLP 支持更加灵活的 Monitor 设置，帮助您自由地监控您想要查看的变量或结果。当 `monitor` 为 `'f#f'` 时表示保存评测结果中 `f#f` 最好的模型。除了字符串之外，fastNLP 也支持将 `monitor` 作为一个函数。这个函数接受一个字典输入（代表评估的结果），返回一个浮点数作为结果。比如我们可以利用评测结果中的 `pre` 和 `rec` 来手动计算 F1 分数：

In [10]:
# 重新初始化
# model = BertNER('bert-base-uncased', len(data_bundle.get_vocab('target')), data_bundle.get_vocab('target'))
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
callbacks = [
    LoadBestModelCallback(),   # 用于在训练结束之后加载性能最好的model的权重
    TorchWarmupCallback()
]

"""
用pre rec重新算一下F值
"""
def monitor_ff(result):
    # F1 = (1 + beta^2) * pre * rec / (beta^2 * pre + rec + 1e-13)
    beta = 1
    pre, rec = result["pre#f"], result["rec#f"]
    return (1 + beta ** 2) * pre * rec / (beta ** 2 * pre + rec + 1e-13)

trainer = Trainer(model=model, train_dataloader=dls['train'], optimizers=optimizer,
                  evaluate_dataloaders=dls['dev'], metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=0, monitor=monitor_ff, fp16=False) # 向 monitor 中传入 monitor_ff 函数
trainer.run()

[09:40:25] INFO     Running evaluator sanity check for 2 batches.              ]8;id=220796;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=692323;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

+++++++++++++++++++++++++++++ Eval. results on Epoch:1, Batch:0 +++++++++++++++++++++++++++++

{
  "f#f": 0.515854,
  "pre#f": 0.560513,
  "rec#f": 0.477785
}

[09:41:22] INFO     The best performance for monitor <function        ]8;id=957685;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=923072;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    monitor_ff at 0x7fb0de601710>:0.5158532592858202                         
                    was achieved in Epoch:1, Global Batch:625. The                           
                    evaluation result:                                                       
                    {'f#f': 0.515854, 'pre#f': 0.560513, 'rec#f':                            
                    0.477785}                                                                

           INFO     Loading best model from buffer with       ]8;id=386040;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=305206;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#115\115]8;;\
                    <function monitor_ff at 0x7fb0de601710>:                                 
                    0.5158532592858202...                                                    

&emsp;&emsp;您可以根据需要在函数中实现更加复杂的功能以满足不同的需求。

#### 7.5 训练过程中，使用不同的测试函数

&emsp;&emsp;如您所见，如果仅通过 `model.evaluate_step` 进行验证的话会有许多局限性，而有时我们会需要不同的函数进行评测，例如在大部分生成任务中，一般使用训练 loss 作为训练过程中的 evaluate ；但同时在训练到一定 epoch 数量之后，会让 model 生成的完整的数据评测 bleu 等。此刻就可能需要两种不同的 `evaluate_fn` 。 fastNLP 提供了 `MoreEvaluateCallback` 来实现这一功能。假如我们想要在上文训练 conll 数据集时使用 `ClassifyFPreRecMetric` 并且用不同的频率进行验证（仅作为演示，实际上这两种 Metric 是用于不同任务的），那么可以进行如下的设置：

In [11]:
from fastNLP import MoreEvaluateCallback
from fastNLP import ClassifyFPreRecMetric

# 重新初始化
# model = BertNER('bert-base-uncased', len(data_bundle.get_vocab('target')), data_bundle.get_vocab('target'))
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
callbacks = [
    LoadBestModelCallback(),   # 用于在训练结束之后加载性能最好的model的权重
    TorchWarmupCallback(),
    MoreEvaluateCallback(
        dls['dev'], # 设置要验证的 dataloader
        metrics={'more_f': ClassifyFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, # metric
        evaluate_every=200, # 每 200 个 batch 触发一次
    )
]

trainer = Trainer(model=model, train_dataloader=dls['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=0, monitor='f#f', fp16=False)
trainer.run()

Output()

{'f#more_f': 0.368421, 'pre#more_f': 0.368421, 'rec#more_f': 0.368421}

[09:41:42] INFO     Running evaluator sanity check for 2 batches.              ]8;id=2342;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=17402;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

...

{'f#more_f': 0.375479, 'pre#more_f': 0.375479, 'rec#more_f': 0.375479}

+++++++++++++++++++++++++++++ Eval. results on Epoch:1, Batch:0 +++++++++++++++++++++++++++++

{
  "f#f": 0.626038,
  "pre#f": 0.651501,
  "rec#f": 0.602491
}

[09:42:44] INFO     The best performance for monitor f#f:0.626038 was ]8;id=725797;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=317286;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    achieved in Epoch:1, Global Batch:625. The                               
                    evaluation result:                                                       
                    {'f#f': 0.626038, 'pre#f': 0.651501, 'rec#f':                            
                    0.602491}                                                                

           INFO     Loading best model from buffer with f#f:  ]8;id=725461;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=596321;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#115\115]8;;\
                    0.626038...                                                              

&emsp;&emsp;可以看到输出中多出了后缀为 `more_f` 的结果。您可以查阅 [MoreEvaluateCallback 的文档](../../api/generated/fastNLP.core.MoreEvaluateCallback.rst) 来了解更多的参数。

#### 7.6 自定义分布式 Metric

&emsp;&emsp;fastNLP 的评测方法支持用户进行自定义，并且也可以实现分布式训练中的评测细节。以下面经过简化的 `Accuracy` 为例，其基类 `Metric` 包含 `backend` 和 `aggregate_when_get_metric` 两个参数，后者表示在最终获得结果时是否需要进行 aggregate 操作。该 Metric  包含两个张量类型的成员 `correct` 和 `total` ，表示正确的预测和样本总数。由于我们需要它实现分布式的功能，因此必须调用 `register_element` 函数来注册该成员，并制定方法 `aggregate_method` 为 `sum` 来表示在分布式训练下需要将多个进程上的结果累加起来，最后再计算出正确率。

&emsp;&emsp;一个 `Metric` 应该包含三个函数：

- `reset`：在验证前调用，用于重置未注册成员的值。在 `Accuracy` 中由于成员均被注册，所以便没有重写该函数
- `update`：对每个 batch 的数据进行处理，比如 `Accuracy` 会比较 pred 和 target 的值，来统计预测正确的数目和样本的总数
- `get_metric`：在一个 dataloader 迭代结束后调用，用于生成结果。`Accuracy` 会计算出这次验证的正确率，然后返回。

In [12]:
import numpy as np
from fastNLP import Metric

class Accuracy(Metric):

    def __init__(self, backend, aggregate_when_get_metric = None):
        super(Accuracy, self).__init__(backend=backend, aggregate_when_get_metric=aggregate_when_get_metric)
        self.register_element(name='correct', value=0, aggregate_method='sum', backend=backend)
        self.register_element(name='total', value=0, aggregate_method="sum", backend=backend)

    def get_metric(self) -> dict:
        evaluate_result = {'acc': round(self.correct.get_scalar() / (self.total.get_scalar() + 1e-12), 6)}
        return evaluate_result

    def update(self, pred, target):

        pred = pred.argmax(axis=-1)

        self.total += np.prod(list(pred.shape)).item()
        self.correct += (target == pred).sum().item()

&emsp;&emsp;而关于分布式 Metric 的构建，我们可以借助 `torch.distributed.gather` 之类的函数来将多卡上的数据聚合到一处，然后进行更新。您可以参考 [SpanFPreRecMetric](../../api/generated/fastNLP.core.SpanFPreRecMetric.rst) 中的 `get_metric()` 方法将多个结果聚合在一起。

#### 7.7 更有效率的Sampler

&emsp;&emsp;fastNLP 还提供了一些其它的 Sampler 或 BatchSampler 来实现更有效率的训练。

- [BucketedBatchSampler](../../api/generated/fastNLP.core.BucketedBatchSampler.rst)：这种 BatchSampler 会首先按照 sample 的长度排序，然后以 batch_size\*num_batch_per_bucket 为一个 `桶` 的大小，数据只会在这个桶内进行组
    合，这样每个 batch 中的 padding 数量会比较少 （因为桶内的数据的长度都接近）。
- [SortedSampler](../../api/generated/fastNLP.core.SortedSampler.rst)：这种 Sampler 会从长到短对数据进行迭代。

&emsp;&emsp;因此，我们可以在训练时使用 `BucketedBatchSampler` 尽可能地减少 padding 数目并随机取样，在不需要随机采样的评估时使用 `SortedSampler` 来达到理论上 padding 最少的效果，提高训练的效率。

In [13]:
from fastNLP import prepare_torch_dataloader, BucketedBatchSampler, SortedSampler

# 还是以上述 NER 任务为例
# 实例化 BucketedBatchSampler
train_batch_sampler = BucketedBatchSampler(
    data_bundle.get_dataset("train"),
    batch_size=32,
    length="input_len",
    num_batch_per_bucket=10,
)
train_dataloader = prepare_torch_dataloader(
    data_bundle.get_dataset("train"),
    batch_sampler=train_batch_sampler
)

# 实例化 SortedSampler
dev_sampler = SortedSampler(data_bundle.get_dataset("dev"), length="input_len")
dev_dataloader = train_dataloader = prepare_torch_dataloader(
    data_bundle.get_dataset("train"),
    sampler=dev_sampler,
)

# 设置训练模型和参数
# model = BertNER('bert-base-uncased', len(data_bundle.get_vocab('target')), data_bundle.get_vocab('target'))
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
callbacks = [
    LoadBestModelCallback(),   # 用于在训练结束之后加载性能最好的model的权重
    TorchWarmupCallback()
]

trainer = Trainer(model=model, train_dataloader=train_dataloader, optimizers=optimizer, 
                  evaluate_dataloaders=dev_dataloader, 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'},
                  device=0, monitor='f#f', fp16=False)
trainer.run()

[09:43:30] INFO     Running evaluator sanity check for 2 batches.              ]8;id=160529;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=180582;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

+++++++++++++++++++++++++++++ Eval. results on Epoch:1, Batch:0 +++++++++++++++++++++++++++++

{
  "f#f": 0.749894,
  "pre#f": 0.742511,
  "rec#f": 0.757425
}

[09:43:51] INFO     The best performance for monitor f#f:0.749894 was ]8;id=2566;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=356757;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    achieved in Epoch:1, Global Batch:217. The                               
                    evaluation result:                                                       
                    {'f#f': 0.749894, 'pre#f': 0.742511, 'rec#f':                            
                    0.757425}                                                                

           INFO     Loading best model from buffer with f#f:  ]8;id=801627;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=801765;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#115\115]8;;\
                    0.749894...                                                              

#### 7.8保存模型

&emsp;&emsp;在 fastNLP 中，可以通过调用 Trainer 和 `save_model` 和 `load_model` 函数来保存和加载模型。参数 `only_state_dict` 会决定在保存或加载时是否仅处理模型的 state_dict，默认为 `True` 。

In [14]:
# model = BertNER('bert-base-uncased', len(data_bundle.get_vocab('target')), data_bundle.get_vocab('target'))
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
callbacks = [
    LoadBestModelCallback(),   # 用于在训练结束之后加载性能最好的model的权重
    TorchWarmupCallback(),
]

trainer = Trainer(model=model, train_dataloader=dls['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=0, monitor='f#f', fp16=False)
trainer.save_model("model", only_state_dict=True)
trainer.load_model("model", only_state_dict=True)

&emsp;&emsp;同样，`Evaluator` 也提供了 `load_model` 接口，这样我们可以使用它来加载已经训练好的模型来进行评估或者测试：

In [15]:
evaluator = Evaluator(model=model, dataloaders=dls['dev'],
                  n_epochs=1, mapping={'first_len': 'seq_len'},
                  device=0)
evaluator.load_model("model", only_state_dict=True)

#### 7.9 断点重训

&emsp;&emsp;fastNLP 提供了断点重训的功能，配合 `CheckpointCallback` 和 `Trainer.run` 的 `resume_from` 参数便可以实现这一功能，使得我们可以从上一次保存的地方重新开始训练（精确到断点的 batch）。下面我们通过一个简单的例子来展示这一功能，可以看到加载前后的数据恰好构成一整个数据集。

In [16]:
from fastNLP import DataSet, CheckpointCallback, logger

# 构造一个简单的数据集
dataset = DataSet({"items": [ [i * 1.0] for i in range(12)]})
dataloader = prepare_dataloader(dataset, batch_size=2, shuffle=True)

# 构造一个简单的模型
class SimpleModule(nn.Module):
    def __init__(self):
        super(SimpleModule, self).__init__()
        self.fc = nn.Linear(1, 1)

    def forward(self, items):
        # 仅仅输出batch的内容
        logger.info(items)
        return {"loss": self.fc(items).sum()}

test_model = SimpleModule()
optimizer = optim.AdamW(test_model.parameters(), lr=2e-5)

callbacks = [
    # 每两个 batch 保存一次
    CheckpointCallback("./checkpoint", every_n_batches=2, save_object="trainer")
]
trainer = Trainer(model=test_model, train_dataloader=dataloader, optimizers=optimizer, 
                  n_epochs=1, device=0, callbacks=callbacks)
trainer.run()

Output()

[09:44:09] INFO     tensor([[9.],                                              ]8;id=646953;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=137153;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [2.]], device='cuda:0')                                          

           INFO     tensor([[7.],                                              ]8;id=58614;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=179282;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [4.]], device='cuda:0')                                          

           INFO     tensor([[ 5.],                                             ]8;id=13894;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=976856;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [11.]], device='cuda:0')                                         

           INFO     tensor([[0.],                                              ]8;id=803569;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=420754;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [3.]], device='cuda:0')                                          

           INFO     tensor([[ 6.],                                             ]8;id=579723;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=257888;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [10.]], device='cuda:0')                                         

[09:44:10] INFO     tensor([[8.],                                              ]8;id=345085;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=895924;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [1.]], device='cuda:0')                                          

&emsp;&emsp;在运行完上述代码后，您会发现在 `checkpoint` 文件夹下出现了我们保存的结果，包含时间、保存的 batch 数目。如果您想要从其中一个断点开始训练，可以运行下面代码：

In [17]:
trainer = Trainer(model=test_model, train_dataloader=dataloader, optimizers=optimizer, 
                  n_epochs=1, device=0,)
# 从 epoch 0 batch 4 开始重训
trainer.run(resume_from="./checkpoint/2022-07-22-15_09_23_162748/trainer-epoch_0-batch_4")

Output()

[09:44:12] INFO     tensor([[ 6.],                                             ]8;id=852159;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=708608;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [10.]], device='cuda:0')                                         

           INFO     tensor([[8.],                                              ]8;id=124938;file:///tmp/ipykernel_6930/21462856.py\21462856.py]8;;\:]8;id=650106;file:///tmp/ipykernel_6930/21462856.py#15\15]8;;\
                            [1.]], device='cuda:0')                                          

&emsp;&emsp;可以看到在迭代一轮的情况下，只进行了两次输出，因为当次训练是从 4 个 batch 之后开始的，只会训练剩下的 2 个 batch。且对比前后的输出可以发现我们取出的后 2 个 batch恰好能够和第一次输出的前 4 个 batch 构成一个完整的数据集，这也是 `可复现` 的含义。除了固定间隔的保存外，`CheckpointCallback` 还支持在每次迭代结束后保存、在出现异常信息时保存、在特定结果变好时保存。更多相关的参数可以阅读 [CheckpointCallback 的文档](../../api/generated/fastNLP.core.CheckpointCallback.rst) 。

#### 7.10 使用huggingface datasets

&emsp;&emsp;fastNLP 还支持 `datasets` 库提供的多种数据集。不过在这种情况下，您需要自己使用 tokenizer 等对数据进行处理。以 `datasets` 提供的 `SetFit/sst2` 数据集为例，我们可以通过下面的代码进行处理。

In [18]:
from fastNLP.transformers.torch import AutoTokenizer # fastNLP 将 transformers 4.11.3 迁移了过来，可以直接进行调用
from fastNLP.io import DataBundle
from datasets import load_dataset

# 加载 torkenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 使用 datasets 加载数据集
sst2 = load_dataset("SetFit/sst2").shuffle(seed=123)["train"]

# 获得验证集和测试集
train_dataset = DataSet.from_pandas(sst2.to_pandas())
train_dataset, val_dataset = train_dataset.split(ratio=0.8, shuffle=True)

print(train_dataset)
# 使用 DataBundle 进行包装和分词
def _process(data):
    # 按照句长 64 进行截断或填充
    data = tokenizer(data, max_length=64, padding="max_length")
    return data

sst2_data_bundle = DataBundle(
    datasets={"train": train_dataset, "val": val_dataset}
)

# 对 text 列使用 _process 函数进行处理
sst2_data_bundle.apply_field_more(
    _process,
    field_name="text",
    num_proc=5,
)
print(sst2_data_bundle.get_dataset("train"))

Using custom data configuration SetFit--sst2-9a0ea0079d8ddf85
Reusing dataset json (/remote-home/shxing/.cache/huggingface/datasets/SetFit___json/SetFit--sst2-9a0ea0079d8ddf85/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /remote-home/shxing/.cache/huggingface/datasets/SetFit___json/SetFit--sst2-9a0ea0079d8ddf85/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-de22e6f9f2c3004c.arrow
Loading cached shuffled indices for dataset at /remote-home/shxing/.cache/huggingface/datasets/SetFit___json/SetFit--sst2-9a0ea0079d8ddf85/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-c5d1cb16e7c8d9b0.arrow
Loading cached shuffled indices for dataset at /remote-home/shxing/.cache/huggingface/datasets/SetFit___json/SetFit--sst2-9a0ea0079d8ddf85/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-ba87bf1c556653d2.arrow


+------------------------------+-------+------------+
| text                         | label | label_text |
+------------------------------+-------+------------+
| this is the sort of low-g... | 0     | negative   |
| the acting in pauline and... | 1     | positive   |
| as dumb and cheesy as the... | 1     | positive   |
| the title , alone , shoul... | 0     | negative   |
| a smart , sassy and excep... | 1     | positive   |
| the slapstick is labored ... | 0     | negative   |
| you ... get a sense of go... | 0     | negative   |
| what sets it apart is the... | 1     | positive   |
| fifty years after the fac... | 1     | positive   |
| in the new guy , even the... | 0     | negative   |
| this sensitive , smart , ... | 1     | positive   |
| with a large cast represe... | 1     | positive   |
| ...                          | ...   | ...        |
+------------------------------+-------+------------+


Output()

Output()

+------------------+-------+------------+------------------+--------------------+--------------------+
| text             | label | label_text | input_ids        | token_type_ids     | attention_mask     |
+------------------+-------+------------+------------------+--------------------+--------------------+
| this is the s... | 0     | negative   | [101, 2023, 2... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| the acting in... | 1     | positive   | [101, 1996, 3... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| as dumb and c... | 1     | positive   | [101, 2004, 1... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| the title , a... | 0     | negative   | [101, 1996, 2... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| a smart , sas... | 1     | positive   | [101, 1037, 6... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| the slapstick... | 0     | negative   | [101, 1996, 1... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| you ... get a... | 0     | negative   | [101, 2017, 1... | [0, 0, 0, 0,

#### 7.11 使用torchmetrics来作为metric

&emsp;&emsp;fastNLP 的评测 metric 同样支持 `torchmetrics` 提供的各种评测方法，不仅仅局限于 fastNLP 的 `Metric` 类型。我们可以构造一个简单的数据集，并使用 `torchmetrics.Accuracy` 进行评估。注意，fastNLP 和 `torchmetrics` 的一些同名评测方法接受的参数名会有些许出入，您可以通过改变模型返回值或通过 `output_mapping` 等参数来进行调整。

In [19]:
from torch.utils.data import Dataset
from torchmetrics import Accuracy as TorchAccuracy # torchmetrics 的 Accuracy

class ArgMaxDataset(Dataset):
    """
    一个预测序列最大值的数据集
    """
    def __init__(self, num_labels=10, data_num=1000):
        self.num_labels = num_labels
        self.data_num = data_num

        self.x = torch.randint(low=-100, high=100, size=[data_num, num_labels]).float()
        self.y = torch.max(self.x, dim=-1)[1]

    def __len__(self):
        return self.data_num

    def __getitem__(self, item):
        return {"x": self.x[item], "y": self.y[item]}

class Classifier(nn.Module):
    """
    简单的分类模型
    """
    def __init__(self, feature_dimension):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(in_features=feature_dimension, out_features=10)
        self.ac1 = nn.ReLU()
        self.output = nn.Linear(in_features=10, out_features=feature_dimension)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.ac1(self.linear1(x))
        x = self.output(x)
        return x

    def train_step(self, x, y):
        x = self(x)
        return {"loss": self.loss_fn(x, y)}

    def evaluate_step(self, x, y):
        x = self(x)
        x = torch.max(x, dim=-1)[1]
        # torchmetrics Accuracy 接受的第一个参数是 preds
        return {"preds": x, "target": y}

dataloaders = prepare_dataloader(
    {
        'train': ArgMaxDataset(10, 100),
        'dev': ArgMaxDataset(10, 100),
    },
    batch_size=4,
)
classifier = Classifier(10)
optimizer = optim.AdamW(classifier.parameters(), lr=2e-5)

trainer = Trainer(model=classifier, train_dataloader=dataloaders['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dataloaders['dev'], metrics={'acc': TorchAccuracy()}, # 这里是 torchmetrics 的 Accuracy 
                  n_epochs=1, device=0)
trainer.run()

[09:50:16] INFO     Running evaluator sanity check for 2 batches.              ]8;id=649680;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=313696;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc": 0.05999999865889549
}

#### 7.12 将预测结果写出到文件

&emsp;&emsp;在训练时，我们有时会希望输出预测结果或是将结果保存到文件中。在 fastNLP 中，我们可以通过三种方法实现该功能。

##### 通过 evaluate_batch_step_fn 实现

&emsp;&emsp;`evaluate_batch_step_fn` 参数接受一个函数，该函数的接受两个参数 `evaluator` 和 `batch` 来处理一个 batch 的数据。对于我们上文定义的 `BertNER` 模型，我们可以自定义一个函数 `output_batch`：

In [20]:
from fastNLP import logger

def output_batch(evaluator, batch):
    logger.info(f"input:{batch['input_ids']}")
    output = evaluator.evaluate_step(batch)
    logger.info(f"prediction:{output['pred']}")

&emsp;&emsp;然后按照正常的流程进行训练即可。训练之后，调用 `Evaluator` 可以实现预测。

In [21]:
from torch import optim
from fastNLP import Trainer, LoadBestModelCallback, TorchWarmupCallback, Evaluator
from fastNLP import SpanFPreRecMetric
model_ner = BertNER('bert-base-uncased', len(data_bundle.get_vocab('target')), data_bundle.get_vocab('target'))
optimizer = optim.AdamW(model_ner.parameters(), lr=2e-5)
callbacks = [
    LoadBestModelCallback(),   # 用于在训练结束之后加载性能最好的model的权重
    TorchWarmupCallback()
]

trainer = Trainer(model=model_ner, train_dataloader=dls['train'], optimizers=optimizer, 
                  evaluate_dataloaders=dls['dev'], 
                  metrics={'f': SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab('target'))}, 
                  n_epochs=1, callbacks=callbacks, 
                  evaluate_input_mapping={'first_len': 'seq_len'}, overfit_batches=0,
                  device=0, monitor='f#f', fp16=False)
trainer.run(5)

evaluator = Evaluator(model=model_ner, dataloaders=dls['test'], 
                      evaluate_input_mapping={'first_len': 'seq_len'}, 
                      evaluate_batch_step_fn=output_batch, # evaluate_batch_step_fn
                      device=0)
evaluator.run(1)

[09:50:30] WARNING  Some weights of the model checkpoint at            ]8;id=796867;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=799227;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1490\1490]8;;\
                    bert-base-uncased were not used when initializing                        
                    BertModel: ['cls.predictions.decoder.weight',                            
                    'cls.predictions.bias',                                                  
                    'cls.predictions.transform.LayerNorm.weight',                            
                    'cls.predictions.transform.dense.weight',                                
                    'cls.predictions.transform.LayerNorm.bias',                              
                    'cls.seq_relationship.bias',                                             
                    'cls.predictions.transform.dense.bias',                                  
                    'cls.seq_relationship.weight']                                           
                    - This IS expected if you are initializing                               
                    BertModel from the checkpoint of a model trained                         
                    on another task or with another architecture (e.g.                       
                    initializing a BertForSequenceClassification model                       
                    from a BertForPreTraining model).                                        
                    - This IS NOT expected if you are initializing                           
                    BertModel from the checkpoint of a model that you                        
                    expect to be exactly identical (initializing a                           
                    BertForSequenceClassification model from a                               
                    BertForSequenceClassification model).                                    

           INFO     All the weights of BertModel were initialized from ]8;id=50181;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py\modeling_utils.py]8;;\:]8;id=776527;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/transformers/torch/modeling_utils.py#1507\1507]8;;\
                    the model checkpoint at bert-base-uncased.                               
                    If your task is similar to the task the model of                         
                    the checkpoint was trained on, you can already use                       
                    BertModel for predictions without further                                
                    training.                                                                

           INFO     Running evaluator sanity check for 2 batches.              ]8;id=887303;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=453413;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

...

           INFO     Loading best model from buffer with f#f:  ]8;id=620717;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=937515;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#115\115]8;;\
                    0.01781...                                                               

Output()

           INFO     input:tensor([[  101,   118,   141,  ...,     0,     0,   ]8;id=69468;file:///tmp/ipykernel_6930/2044833677.py\2044833677.py]8;;\:]8;id=770812;file:///tmp/ipykernel_6930/2044833677.py#4\4]8;;\
                    0],                                                                      
                            [  101,   156,  9244,  ...,     0,     0,     0],                
                            [  101, 11896,  3309,  ...,     0,     0,     0],                
                            ...,                                                             
                            [  101, 15919,  1116,  ...,     0,     0,     0],                
                            [  101,  1398,  1300,  ...,     0,     0,     0],                
                            [  101,   118,   141,  ...,     0,     0,                        
                    0]], device='cuda:0')                                                    

           INFO     prediction:tensor([[0, 4, 8,  ..., 8, 8, 8],              ]8;id=742913;file:///tmp/ipykernel_6930/2044833677.py\2044833677.py]8;;\:]8;id=589604;file:///tmp/ipykernel_6930/2044833677.py#6\6]8;;\
                            [0, 0, 0,  ..., 8, 8, 8],                                        
                            [7, 7, 6,  ..., 8, 8, 8],                                        
                            ...,                                                             
                            [0, 0, 0,  ..., 8, 8, 8],                                        
                            [0, 0, 0,  ..., 0, 0, 0],                                        
                            [0, 4, 8,  ..., 8, 8, 8]], device='cuda:0')                      

{}

##### 通过 Metric 实现

&emsp;&emsp;第二种方法是利用 Metric 进行输出或者保存。虽然 Metric 是用于评测的类，但其 **收集每个 batch 的数据 - 最后进行统一运算** 的过程也可以用于数据结果或者保存函数。并且由于 fastNLP 同样支持分布式的 Metric，因此该方法也可以在分布式训练中有相当不错的表现。比如我们想要实现一个将所有预测结果写入 `output.txt` 的功能呢，可以按如下方式编写：

In [22]:
class OutputSaver(Metric):
    def __init__(self, tag_vocab, filename):
        super(OutputSaver, self).__init__()
        self.tag_vocab = tag_vocab
        self.filename = filename
        self.words_list = []
        self.targets_list = []
        self.labels_list = []

    def reset(self):
        self.words_list = []
        self.targets_list = []
        self.labels_list = []

    def update(self, pred, raw_words, target):
        for words, single_target, output in zip(raw_words, target, pred):
            # 收集数据、预测标签和真实标签
            # 通过词表转换为字符串形式的结果
            labels = [data_bundle.get_vocab("target").idx2word[idx] for idx in output[:len(words)].tolist() ]
            # 由于我们在前文使用 set_ignore 将 raw_target 忽略了，因此这里是无法获取到 raw_target 的，只能手动转换
            # 当然这只是演示用，实际请根据需求调整是否忽略数据的某一列
            raw_target = [data_bundle.get_vocab("target").idx2word[idx] for idx in single_target[:len(words)].tolist() ]
            self.words_list.append(words)
            self.targets_list.append(raw_target)
            self.labels_list.append(labels)

    def get_metric(self):
        with open(self.filename, "w") as f:
            # 逐行写入文件
            for words, targets, labels in zip(self.words_list, self.targets_list, self.labels_list):
                f.write(" ".join(words) + "\n")
                f.write(" ".join(labels) + "\n")
                f.write(" ".join(targets) + "\n")
                f.write("\n")

evaluator = Evaluator(model=model, dataloaders=dls["test"], 
                      device=0, n_epochs=1,
                      metrics={"output": OutputSaver(data_bundle.get_vocab("target"), "output.txt")})
evaluator.run()

Output()

[09:50:47] WARNING  Metric:output returns None when getting metric results.  ]8;id=245516;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/evaluator.py\evaluator.py]8;;\:]8;id=641709;file:///remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/controllers/evaluator.py#529\529]8;;\

{}

&emsp;&emsp;如果您想在分布式训练中实现这一功能，那么您也可以借助  `all_gather_object` 或 `all_gather` 并行操作进行输出：

```python
import torch.distributed as dist
from fastNLP import Metric

class Output(Metric):

    def __init__(self, backend, aggregate_when_get_metric = None):
        super(Output, self).__init__(backend=backend, aggregate_when_get_metric=aggregate_when_get_metric)
        self.preds = []

    def get_metric(self) -> dict:
        gathered_preds = [None for _ in range(dist.get_world_size())]
        # 收集其它 rank 的数据
        torch.distributed.all_gather_object(gathered_preds, self.preds)
        if dist.get_rank() == 0:
            # 仅在 rank 0 输出
            for rank, preds in gathered_preds:
                print("prediction of rank", rank)
                for input_ids, pred in preds:
                    print("input_ids", input_ids)
                    print("prediction", pred)

    def update(self, input_ids, pred):
        # 添加到 preds 中
        self.preds.append((input_ids, pred))

    def reset(self):
        self.preds = []
```

##### 通过 evaluate_fn 实现

&emsp;&emsp;第三种方法是通过指定 `evaluate_fn` 实现，即在模型中添加一个函数，然后进行训练，相当于将第一种方法的 `output_batch` 函数作为模型的成员函数。不过这种方法会涉及对模型的修改，因此不如前两种方法灵活，这里就不做赘述了。

#### 7.13 混合 dataset 训练

&emsp;&emsp;在某些任务中，可能需要您使用多个数据集进行训练，而 fastNLP 也提供了这一功能，通过调用 `fastNLP.core.dataloaders.MixDataLoader` 实现。

In [23]:
from fastNLP.core.dataloaders import MixDataLoader
from fastNLP import DataSet

# 生成顺序数据集，并且数据长度各不相同
datasets = {
    "first": DataSet({"items": [ [i, i, i] for i in range(4)]}),
    "second": DataSet({"items": [ [i, i] for i in range(4, 8)]}),
}

print("'sequential' 模式表示按顺序取数据：")
mix_dataloader = MixDataLoader(datasets, batch_size=2, mode='sequential')
for batch in mix_dataloader:
    print(batch)

print("'mix' 模式表示将数据混合后随机取样：")
mix_dataloader = MixDataLoader(datasets, batch_size=2, mode='mix')
for batch in mix_dataloader:
    print(batch)

print("'polling' 模式表示依次从每个数据集中轮询取数据：")
mix_dataloader = MixDataLoader(datasets, batch_size=2, mode='polling')
for batch in mix_dataloader:
    print(batch)


'sequential' 模式表示按顺序取数据：
{'items': tensor([[0, 0, 0],
        [1, 1, 1]])}
{'items': tensor([[2, 2, 2],
        [3, 3, 3]])}
{'items': tensor([[4, 4],
        [5, 5]])}
{'items': tensor([[6, 6],
        [7, 7]])}
'mix' 模式表示将数据混合后随机取样：
{'items': tensor([[4, 4, 0],
        [0, 0, 0]])}
{'items': tensor([[1, 1, 1],
        [5, 5, 0]])}
{'items': tensor([[2, 2, 2],
        [3, 3, 3]])}
{'items': tensor([[6, 6],
        [7, 7]])}
'polling' 模式表示依次从每个数据集中轮询取数据：
{'items': tensor([[0, 0, 0],
        [1, 1, 1]])}
{'items': tensor([[4, 4],
        [5, 5]])}
{'items': tensor([[2, 2, 2],
        [3, 3, 3]])}
{'items': tensor([[6, 6],
        [7, 7]])}


/remote-home/shxing/anaconda3/envs/fastnlp/lib/python3.7/site-packages/fastNLP/core/samplers/mix_sampler.py:239: UserWarning: you are shuffling a 'array' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(total_index)


In [24]:
# 生成顺序数据集，并且数据长度各不相同、总长也不同
datasets = {
    "first": DataSet({"items": [ [i, i, i] for i in range(4)]}),
    "second": DataSet({"items": [ [i, i] for i in range(4, 10)]}),
}

print("ds_ratio='truncate_to_least' 会舍弃较大数据集的部分数据：")
mix_dataloader = MixDataLoader(datasets, batch_size=2, ds_ratio='truncate_to_least')
for batch in mix_dataloader:
    print(batch)

print("ds_ratio='pad_to_most' 会对较小的数据集进行重采样：")
mix_dataloader = MixDataLoader(datasets, batch_size=2, ds_ratio='pad_to_most')
for batch in mix_dataloader:
    print(batch)

ds_ratio='truncate_to_least' 会舍弃较大数据集的部分数据：
{'items': tensor([[0, 0, 0],
        [1, 1, 1]])}
{'items': tensor([[2, 2, 2],
        [3, 3, 3]])}
{'items': tensor([[4, 4],
        [5, 5]])}
{'items': tensor([[6, 6],
        [7, 7]])}
ds_ratio='pad_to_most' 会对较小的数据集进行重采样：
{'items': tensor([[0, 0, 0],
        [1, 1, 1]])}
{'items': tensor([[2, 2, 2],
        [3, 3, 3]])}
{'items': tensor([[0, 0, 0],
        [1, 1, 1]])}
{'items': tensor([[4, 4],
        [5, 5]])}
{'items': tensor([[6, 6],
        [7, 7]])}
{'items': tensor([[8, 8],
        [9, 9]])}


&emsp;&emsp;您可以查阅 [MixDataLoader 的文档](../../api/generated/fastNLP.core.MixDataLoader.rst) 来详细了解它的用法。

#### 7.14 logger的使用

&emsp;&emsp;fastNLP 的 `logger` 模块可以帮助您打印训练中的各种信息，让您一目了然地了解训练过程。其中 `rank_zero_warning` 可以在分布式训练中使用，只会在 rank 0 上输出警告，`warning_once` 则表示这条警告只会输出一次。`print` 函数则会将输入的内容输出为 INFO 。

In [25]:
from fastNLP import logger, print

logger.info("information")
logger.warn("warning")
logger.rank_zero_warning("warning")
logger.warning_once("warning")
logger.error("error")

print("print as info")

[09:50:53] INFO     information                                                ]8;id=804116;file:///tmp/ipykernel_6930/428786443.py\428786443.py]8;;\:]8;id=614645;file:///tmp/ipykernel_6930/428786443.py#3\3]8;;\

           WARNING  warning                                                    ]8;id=940360;file:///tmp/ipykernel_6930/428786443.py\428786443.py]8;;\:]8;id=184056;file:///tmp/ipykernel_6930/428786443.py#4\4]8;;\

           WARNING  warning                                                    ]8;id=91051;file:///tmp/ipykernel_6930/428786443.py\428786443.py]8;;\:]8;id=168002;file:///tmp/ipykernel_6930/428786443.py#5\5]8;;\

           WARNING  warning                                                    ]8;id=748837;file:///tmp/ipykernel_6930/428786443.py\428786443.py]8;;\:]8;id=772001;file:///tmp/ipykernel_6930/428786443.py#6\6]8;;\

           ERROR    error                                                      ]8;id=739810;file:///tmp/ipykernel_6930/428786443.py\428786443.py]8;;\:]8;id=745954;file:///tmp/ipykernel_6930/428786443.py#7\7]8;;\

           INFO     print as info                                              ]8;id=775202;file:///tmp/ipykernel_6930/428786443.py\428786443.py]8;;\:]8;id=964443;file:///tmp/ipykernel_6930/428786443.py#9\9]8;;\

#### 7.15 Metric以下划线开头的指标不会打印

&emsp;&emsp;`fastNLP` 中的 `progress_bar` 会自动过滤掉以 `_` 开头的评价指标，可以过滤掉用户不希望展示在终端的指标，在下面我们自定义的 `metric` 中，`_hidden` 是不会打印的，`progress_bar` 会从列表中过滤，请看示例：

In [1]:
from fastNLP import Metric
class Accuracy(Metric):

    def __init__(self, backend="torch", aggregate_when_get_metric = None):
        super(Accuracy, self).__init__(backend=backend, aggregate_when_get_metric=aggregate_when_get_metric)
        self.register_element(name='correct', value=0, aggregate_method='sum', backend=backend)
        self.register_element(name='total', value=0, aggregate_method="sum", backend=backend)

    def get_metric(self) -> dict:
        evaluate_result = {'acc': round(self.correct.get_scalar() / (self.total.get_scalar() + 1e-12), 6),
                           '_hidden': round(self.correct.get_scalar() / (self.total.get_scalar() + 1e-12), 6),
                           'hidden': round(self.correct.get_scalar() / (self.total.get_scalar() + 1e-12), 6)}
        return evaluate_result

    def update(self, pred, target):

        pred = pred.argmax(axis=-1)

        self.total += np.prod(list(pred.shape)).item()
        self.correct += (target == pred).sum().item()

In [2]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset

from fastNLP import prepare_dataloader, Trainer


class ArgMaxDataset(Dataset):
    """
    一个预测序列最大值的数据集
    """
    def __init__(self, num_labels=10, data_num=1000):
        self.num_labels = num_labels
        self.data_num = data_num

        self.x = torch.randint(low=-100, high=100, size=[data_num, num_labels]).float()
        self.y = torch.max(self.x, dim=-1)[1]

    def __len__(self):
        return self.data_num

    def __getitem__(self, item):
        return {"x": self.x[item], "y": self.y[item]}

class Classifier(nn.Module):
    """
    简单的分类模型
    """
    def __init__(self, feature_dimension):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(in_features=feature_dimension, out_features=10)
        self.ac1 = nn.ReLU()
        self.output = nn.Linear(in_features=10, out_features=feature_dimension)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.ac1(self.linear1(x))
        x = self.output(x)
        return x

    def train_step(self, x, y):
        x = self(x)
        return {"loss": self.loss_fn(x, y)}

    def evaluate_step(self, x, y):
        x = self(x)
        x = torch.max(x, dim=-1)[1]
        return {"pred": x, "target": y}

dataloaders = prepare_dataloader(
    {
        'train': ArgMaxDataset(10, 100),
        'dev': ArgMaxDataset(10, 100),
    },
    batch_size=4,
)
classifier = Classifier(10)
optimizer = optim.AdamW(classifier.parameters(), lr=2e-5)

trainer = Trainer(model=classifier, train_dataloader=dataloaders['train'], optimizers=optimizer,
                  evaluate_dataloaders=dataloaders['dev'], metrics={'acc': Accuracy()},
                  n_epochs=1, device=0)
trainer.run()

[19:48:09] INFO     Running evaluator sanity check for 2 batches.              ]8;id=67801;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=104711;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc#acc": 0.32,
  "hidden#acc": 0.32
}